In [2]:
try:
    import unicodecsv
except:
    !pip install unicodecsv
    import unicodecsv

In [3]:
#This will load the csv
import csv
def loadCsv(filename):
    info = open(filename, "rb")
    has_header = csv.Sniffer().has_header(info.read(1024))
    info.seek(0)
    incsv = csv.reader(info)
    if has_header:
        next(incsv)  #Skip header
    dataset = list(incsv)
    return dataset
filename = 'train_set_x.csv'
dataset = loadCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(dataset))
print(dataset[18])

Loaded data file train_set_x.csv with 276517 rows
['18', 'Was f\xc3\xbcr eine Mail Was soll da drin stehen']


In [4]:
#get Id from list object
def getId(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    return new_line[0]

In [5]:
#get Utterance from list object
def getUtt(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    return new_line[1]

In [7]:
#Separate by class
#Assuming last index has the class nb
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

separated = separateByClass(dataset)
print(separated)

{'1': [['0', 'hello betch', '1']], '2': [['1', 'yooo', '2']]}
